In [1]:
import torch
import gc
import pandas as pd
import math
from transformers import AutoProcessor, SeamlessM4Tv2ForTextToText
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
SOURCE_LANGUAGE = "eng"
TARGET_LANGUAGES = [
    "spa",
    "cmn",
    "kor",
    "arb",
    "ell",
    "swh",
    "amh",
    "vie",
]

assert torch.cuda.is_available(), "expected cuda"
device = "cuda"

model = SeamlessM4Tv2ForTextToText.from_pretrained("facebook/seamless-m4t-v2-large").to(device)
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# write 200 shuffled entries
df = pd.read_csv(f"/data/healthy-ml/scratch/maanas/multilingual_jailbreak_scratch/multilingual-latent-adversarial-training/tasks/harmbench/data/clean_behaviors.csv")
df = df.sample(n=200).reset_index(drop=True)
df.to_csv(f"clean_behaviors_{SOURCE_LANGUAGE}.csv", index=False)

In [ ]:
for target_language in TARGET_LANGUAGES:

    def translate_batch(texts: list[str], targ_lang: str) -> list[str]:
        text_inputs = processor(text = texts, src_lang = SOURCE_LANGUAGE, return_tensors="pt").to(device)
        text_outputs = model.generate(**text_inputs, tgt_lang = targ_lang, num_beams=4, do_sample=True)
        return processor.batch_decode(text_outputs, skip_special_tokens=True)


    # df = pd.read_csv("/data/healthy-ml/scratch/maanas/multilingual_jailbreak_scratch/multilingual-latent-adversarial-training/tasks/harmbench/data/harmbench_data/behavior_datasets/harmbench_behaviors_text_all.csv")

    # english_prompts = df["Behavior"].tolist()
    # translated_prompts = [translate_batch([prompt], target_language)[0] for prompt in tqdm(english_prompts)]

    # df["Behavior"] = translated_prompts
    # df.to_csv(f"harmbench_text_{target_language}.csv", index=False)

    # Do benign ones
    df = pd.read_csv(f"clean_behaviors_{SOURCE_LANGUAGE}.csv")
    english_goals = df["goal"].tolist()
    translated_goals = [translate_batch([goal], target_language)[0] for goal in tqdm(english_goals)]

    english_targets = df["target"].tolist()
    translated_targets = [translate_batch([target], target_language)[0] for target in tqdm(english_targets)]

    df["goal"] = translated_goals
    df["target"] = translated_targets
    df.to_csv(f"clean_behaviors_{target_language}.csv", index=False)




















































































































































































































































































































































































































100%|██████████| 400/400 [04:31<00:00,  1.47it/s]

















































































































































































































































































































































































































100%|██████████| 400/400 [04:17<00:00,  1.55it/s]



































































































